In [ ]:
import numpy as np
from PIL import Image, ImageDraw
import os
import cv2
import time
import tensorflow as tf

from detector import Detector

In [ ]:
MODEL_PATH = 'model.pb'
OUTPUT_NAMES = [
    'boxes', 'scores', 'num_boxes',
    'keypoint_heatmaps', 'segmentation_masks',
    'keypoint_scores', 'keypoint_positions'
]

with tf.gfile.GFile(MODEL_PATH, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

graph = tf.Graph()
with graph.as_default():
    tf.import_graph_def(graph_def, name='import')

input_image = graph.get_tensor_by_name('import/images:0')
output_ops = {n: graph.get_tensor_by_name(f'import/{n}:0') for n in OUTPUT_NAMES}
sess = tf.Session(graph=graph)

# Get an image

In [ ]:
path = '/home/dan/datasets/COCO/images/val2017/'
i = os.listdir(path)[600]
path = path + i
image_array = cv2.imread(path)
image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
image = Image.fromarray(image_array)
image = image.resize((640, 640))
image

In [ ]:
outputs = sess.run(output_ops, feed_dict={input_image: np.expand_dims(np.array(image), 0)})
outputs = {n: v[0] for n, v in outputs.items()}

# Show detections

In [ ]:
outputs['keypoint_heatmaps'][:, :, 0].shape

In [ ]:
def draw_everything(image, outputs):

    image_copy = image.copy()
    image_copy.putalpha(255)
    draw = ImageDraw.Draw(image_copy, 'RGBA')
    width, height = image_copy.size
    scaler = np.array([height, width, height, width])
    
    n = outputs['num_boxes']
    boxes = scaler * outputs['boxes']
    for box in boxes:
        ymin, xmin, ymax, xmax = box
        draw.rectangle([(xmin, ymin), (xmax, ymax)], outline='red')
    
    mask = outputs['keypoint_heatmaps'][:, :, 0]#outputs['segmentation_masks']
    m, M = mask.min(), mask.max()
    mask = (mask - m)/(M - m)
    mask = np.expand_dims(mask, 2)
    color = np.array([255, 255, 255])
    mask = Image.fromarray((mask*color).astype('uint8'))
    mask.putalpha(mask.convert('L'))
    mask = mask.resize((width, height))
    image_copy.alpha_composite(mask)
    return image_copy

In [ ]:
draw_everything(image, outputs)

In [ ]:
segmentation_masks = results['segmentation_masks'][0]
segmentation_masks = results['segmentation_masks'][0]

In [ ]:
m = segmentation_masks.min()
M = segmentation_masks.max()
segmentation_masks = (segmentation_masks - m)/(M - m)
segmentation_masks = Image.fromarray((segmentation_masks*255.0).astype('uint8'))
segmentation_masks

# Measure speed

In [ ]:
times = []
for _ in range(110):
    start = time.perf_counter()
    boxes, labels, scores = detector(image_array, score_threshold=0.25)
    times.append(time.perf_counter() - start)
    
times = np.array(times)
times = times[10:]
print(times.mean(), times.std())